## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [33]:
#Importing helpful Libraries
import imageio
import os
import pandas as pd
import numpy as np
from skimage.transform import resize
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util

Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [ ]:
#I am not using colab abd referring to my local

In [3]:
project_path = ""

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [4]:
from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [5]:
from zipfile import ZipFile
with ZipFile(project_path+'test.zip', 'r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [6]:
from zipfile import ZipFile
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

Repeat the same step for labels.csv.zip

In [7]:
from zipfile import ZipFile
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
  z.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [34]:
labels_df = pd.read_csv('labels.csv')

In [4]:
labels_df.head(4)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick


In [5]:
labels_df.shape

(10222, 2)

### Print the count of each category of Dogs given in the dataset



In [11]:
class_df.groupby(by='breed').agg({'id':'count'})

,id
breed,
affenpinscher,80
afghan_hound,116
african_hunting_dog,86
airedale,107
american_staffordshire_terrier,74
appenzeller,78
australian_terrier,102
basenji,110
basset,82


### Get one-hot encodings of labels

In [35]:
dogbreeds = labels_df.breed.unique()

In [36]:
# Label encoding
class_label = {}
index = 0
for breed in dogbreeds:
    class_label[breed] = index
    index += 1
class_label

{'boston_bull': 0,
 'dingo': 1,
 'pekinese': 2,
 'bluetick': 3,
 'golden_retriever': 4,
 'bedlington_terrier': 5,
 'borzoi': 6,
 'basenji': 7,
 'scottish_deerhound': 8,
 'shetland_sheepdog': 9,
 'walker_hound': 10,
 'maltese_dog': 11,
 'norfolk_terrier': 12,
 'african_hunting_dog': 13,
 'wire-haired_fox_terrier': 14,
 'redbone': 15,
 'lakeland_terrier': 16,
 'boxer': 17,
 'doberman': 18,
 'otterhound': 19,
 'standard_schnauzer': 20,
 'irish_water_spaniel': 21,
 'black-and-tan_coonhound': 22,
 'cairn': 23,
 'affenpinscher': 24,
 'labrador_retriever': 25,
 'ibizan_hound': 26,
 'english_setter': 27,
 'weimaraner': 28,
 'giant_schnauzer': 29,
 'groenendael': 30,
 'dhole': 31,
 'toy_poodle': 32,
 'border_terrier': 33,
 'tibetan_terrier': 34,
 'norwegian_elkhound': 35,
 'shih-tzu': 36,
 'irish_terrier': 37,
 'kuvasz': 38,
 'german_shepherd': 39,
 'greater_swiss_mountain_dog': 40,
 'basset': 41,
 'australian_terrier': 42,
 'schipperke': 43,
 'rhodesian_ridgeback': 44,
 'irish_setter': 45,
 'a

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [8]:
img_df = {'image': [],'label': []}

for data in labels_df.values:
    filename = os.path.join('train', data[0]+'.jpg')
    breedname = data[1]
    im = imageio.imread(filename)
    img = resize(im, (64, 64, 3))
    img_df['image'].append(img)
    img_df['label'].append(class_label[breedname])

C:\AIML\Anaconda3\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\AIML\Anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [9]:
x_train = np.array(img_df['image'])
y_train = img_df['label']

Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [10]:
x_train = x_train.astype('float32')
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [11]:
x_train_data, x_test_data, y_train_data, y_test_data = train_test_split(x_train, y_train, test_size=0.20, random_state=1)

### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [12]:
test_img = pd.read_csv('sample_submission.csv')

In [13]:
test_img.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
1,00102ee9d8eb90812350685311fe5890,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
2,0012a730dfa437f5f3613fb75efcd4ce,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
3,001510bc8570bbeee98c8d80c8a95ec1,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
4,001a5f3114548acdefa3d4da05474c2e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333


Run the below code to load the test image files in x_test_feature

In [14]:
x_test_feature = []

for data in test_img.id:
    filename = os.path.join('test', str(data)+'.jpg')
    im = imageio.imread(filename)
    img = resize(im, (64, 64, 3))
    x_test_feature.append(img)

Normalize the test data and convert it into 4 dimensions

In [15]:
x_test_feature = (x_test_feature-mean)/(std+1e-7)

In [16]:
x_test_feature.shape

(10357, 64, 64, 3)

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [24]:
import keras
from keras.models import Sequential
from keras import regularizers
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape, BatchNormalization, Input
from keras.preprocessing.image import ImageDataGenerator

In [18]:
x_train_data.shape[1:]

(64, 64, 3)

In [19]:
weight_decay = 1e-4

conv_layers = [
    Conv2D(128, (5,5), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=(64, 64, 3)),
    Activation('relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)),
    Activation('relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2))
]
dense_layers = [
    Flatten(),
    Dense(256),
    Activation('relu'),
    BatchNormalization(),
    Dense(120),
    Activation('softmax')
]

model = Sequential(conv_layers + dense_layers)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
callback_list = [early_stopping]

### Use batch_size = 128 and epochs = 10 and execute the model

In [21]:
BATCH_SIZE = 128
EPOCHS = 10

model.fit(x_train_data,
          y_train_data, 
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          verbose=True,
          validation_data=(x_test_data, y_test_data),
          callbacks=callback_list
)

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 1759s 215ms/step - loss: 4.7078 - acc: 0.0544 - val_loss: 4.9661 - val_acc: 0.0372
Epoch 2/10
8177/8177 [==============================] - 1589s 194ms/step - loss: 3.7410 - acc: 0.1679 - val_loss: 4.5357 - val_acc: 0.0631
Epoch 3/10
8177/8177 [==============================] - 1592s 195ms/step - loss: 2.3597 - acc: 0.5147 - val_loss: 4.6272 - val_acc: 0.0553
Epoch 4/10
8177/8177 [==============================] - 1600s 196ms/step - loss: 0.6468 - acc: 0.9407 - val_loss: 4.7829 - val_acc: 0.0553
Epoch 5/10
8177/8177 [==============================] - 1574s 192ms/step - loss: 0.0930 - acc: 0.9987 - val_loss: 4.7089 - val_acc: 0.0675
Epoch 6/10
8177/8177 [==============================] - 1574s 192ms/step - loss: 0.0344 - acc: 0.9995 - val_loss: 4.6814 - val_acc: 0.0660
Epoch 7/10
8177/8177 [==============================] - 1577s 193ms/step - loss: 0.0295 - acc: 0.9993 - val_loss: 4.76

In [22]:
model.evaluate(x_test_data, y_test_data)

2045/2045 [==============================] - 79s 38ms/step


[4.852544971083662, 0.06845965769442486]

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [37]:
breed_df = labels_df.groupby(by='breed').agg({'id':'count'})
max(breed_df.id)

126

In [38]:
MAX_DATA = 125

img_df = {'image': [],'label': []}

for index, breed in breed_df.itertuples():
    nooffiles = breed
    breedname = index
    nooffiles_togen = MAX_DATA - nooffiles
    breed_data = labels_df[labels_df.breed == breedname]
    gen_files = 0
    for data in breed_data.values:
        filename = os.path.join('train', data[0]+'.jpg')
        im = imageio.imread(filename)
        img = resize(im, (128, 128, 3))
        img_df['image'].append(img)
        img_df['label'].append(class_label[breedname])
        if gen_files < nooffiles_togen:
            #Randon Rotation
            random_degree = random.uniform(-25, 25)
            transformed_image = sk.transform.rotate(img, random_degree)
            img_df['image'].append(transformed_image)
            img_df['label'].append(class_label[breedname])
            gen_files += 1
            #Horizontal Flip
            transformed_image = img[:, ::-1]
            img_df['image'].append(transformed_image)
            img_df['label'].append(class_label[breedname])
            gen_files += 1

C:\AIML\Anaconda3\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\AIML\Anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [73]:
len(img_df['label'])

15060

In [74]:
pd.DataFrame(img_df['label']).groupby(by=0).agg({0:'count'})

,0
0,
0,125
1,126
2,125
3,125
4,125
5,125
6,125
7,126
8,126


### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [39]:
X = np.array(img_df['image'])
Y = img_df['label']

X = X.astype('float32')
mean = np.mean(X,axis=(0,1,2,3))
std = np.std(X,axis=(0,1,2,3))
X = (X-mean)/(std+1e-7)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=1)

In [40]:
train_generator = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True
)

train_generator.fit(x_train)

In [41]:
value_generator = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True
)

value_generator.fit(x_test)

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [ ]:
BATCH_SIZE = 500
EPOCHS = 10

model.fit_generator(
    train_generator.flow(x_train, y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=EPOCHS*len(x_train)/BATCH_SIZE,
    epochs=EPOCHS,
    verbose=1,
    shuffle=True,
    validation_data=value_generator.flow(x_test, y_test),
    validation_steps=EPOCHS*len(x_test)/BATCH_SIZE,
    callbacks=callback_list
)

Epoch 1/10
  3/240 [..............................] - ETA: 11:26:48 - loss: 4.8231 - acc: 0.0547

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [26]:
from keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg')

Print the summary of the base_model

In [2]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [19]:
base_model.output_shape[1:]

(512,)

In [28]:
dense_layers = [
#    Flatten(input_shape=(base_model.output_shape[1:])),
    Dense(1024),
    Activation('relu'),
    Dense(256),
    Activation('relu'),
    Dense(120),
    Activation('softmax')
]

new_model = Sequential()
for layer in base_model.layers:
    new_model.add(layer)
for layer in dense_layers:
    new_model.add(layer)

In [3]:
#Flatten LAyer
base_model.add_update(Flatten())
# Fully Connected Layer 1
base_model.add_update(Dense(1024))
base_model.add_update(Activation('relu'))
# Fully Connected Layer 2
base_model.add_update(Dense(256))
base_model.add_update(Activation('relu'))

# Prediction Layer
base_model.add_update(Dense(output_dim=120, init='he_normal', bias=True))
base_model.add_update(Activation('softmax'))

C:\AIML\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=120, kernel_initializer="he_normal", use_bias=True)`
  # This is added back by InteractiveShellApp.init_path()


In [29]:
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, None, None, 256)   295168    
__________

### Make all the layers in the base_model (VGG16) to be non-trainable

In [30]:
for layer in new_model.layers:
    layer.trainable = False

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [42]:
new_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
callback_list = [early_stopping]

In [ ]:
BATCH_SIZE = 128
EPOCHS = 10

new_model.fit_generator(
    train_generator.flow(x_train, y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=EPOCHS*len(x_train)/BATCH_SIZE,
    epochs=EPOCHS,
    verbose=1,
    shuffle=True,
    validation_data=value_generator.flow(x_test, y_test),
    validation_steps=EPOCHS*len(x_test)/BATCH_SIZE,
    callbacks=callback_list
)

Epoch 1/10
942/941 [==============================] - 15277s 16s/step - loss: 4.8355 - acc: 0.0085 - val_loss: 4.8372 - val_acc: 0.0091
Epoch 2/10
942/941 [==============================] - 15462s 16s/step - loss: 4.8355 - acc: 0.0086 - val_loss: 4.8347 - val_acc: 0.0084
Epoch 3/10
684/941 [====================>.........] - ETA: 1:05:44 - loss: 4.8357 - acc: 0.0089